In [51]:
import numpy as np
import pandas as pd
import seaborn as sns
import subprocess
import matplotlib.pyplot as plt
import sys
import os
from fnmatch import fnmatch

In [52]:
def listfiles(root, pattern):
  for path, subdirs, files in os.walk(root):
    for name in files:
      if fnmatch(name, pattern):
        yield os.path.join(path, name)

# EXPERIMENTOS CUANTITATIVOS

## Diferencia de resultados

Tenemos 3 metodos:
   - CMM
   - WP
   - WP con Laplace

Para cada método, ejecutamos un conjunto de test supervisados y comparamos el modulo del error obtenido en cada uno.
Para CMM usamos los tests de la catedra. En los otros dos, los generamos utilizando los inputs de la catedra, pero outputs calculados a mano.

In [79]:
def correr_metodo(inputPath, outputPath, metodo):
    result = subprocess.run([f'../cmake-build-debug/tp1', inputPath, outputPath, metodo],
                            stdout=subprocess.PIPE, encoding='ascii')
    return result

### CMM

In [69]:
#Runnea todos los test de la catedra y guarda las salidas
outFolder = './cuantitativo_diff/'
def runCMMDifference():
    for fname in listfiles('../tests', '*.in'):
        name = fname.split('/')[-1]
        #print(name[-1])
        correr_metodo(fname, outFolder + name.replace(".in", ".out"), '0')

In [21]:
#runCMMDifference();

# EXPERIMENTOS CUALITATIVOS

En este experimento vamos a ejecutar un set de datos de la temporada 2016 de la nba, sin la ultima fecha. Despues de eso, vamos a ver que ranking (de cada metodo) se ajusta mejor a los resultados reales en la ultima fecha.

In [85]:
def runCompararConDatosReales():
    for i in range(0, 2):
        correr_metodo('../data/nba_2016_scores_without_lastdate.dat', './cualitativo_sin_ultima_fecha.csv', str(i))

exportPath = 'cualitativo_sin_ultima_fecha.csv'

correr_metodo('../data/nba_2016_scores_without_lastdate.dat', exportPath, '0')

CompletedProcess(args=['../cmake-build-debug/tp1', '../data/nba_2016_scores_without_lastdate.dat', 'cualitativo_sin_ultima_fecha.csv', '0'], returncode=0, stdout='')

In [344]:
def armarTabla(scoresPath):
    #Busco el resultado
    scores = pd.read_csv(scoresPath, names=['Score'])
    #Busco los nombres de los equipos
    equipos = pd.read_csv('../data/nba_2016_teams.csv', names=['ID', 'Team'])
    #Join de los df
    rankings = equipos.join(scores).sort_values(by='Score', ascending=False)

    ultimosPartidos = pd.read_csv('../data/nba_2016_scores_lastdate.csv')
    ultimosPartidos['Acerto'] = 'No'

    for partido in ultimosPartidos.itertuples():
        team1 = rankings[rankings['ID'] == partido.equipo1]
        team2 = rankings[rankings['ID'] == partido.equipo2]
        score1 = team1['Score']
        score2 = team2['Score']

        #Guardo el nombre de los equipos
        ultimosPartidos.at[partido.Index, 'Equipo 1'] = team1['Team'].values[0]
        ultimosPartidos.at[partido.Index, 'Equipo 2'] = team2['Team'].values[0]
        #Guardo el score que saco en el ranking cada equipo
        ultimosPartidos.at[partido.Index, 'Score equipo 1'] = team1['Score'].values[0]
        ultimosPartidos.at[partido.Index, 'Score equipo 2'] = team2['Score'].values[0]
        #Guardo el resultado del partido en 1 sola columna
        ultimosPartidos.at[partido.Index, 'Resultado'] = "%s - %s" %(partido.puntaje1, partido.puntaje2)

        if (partido.puntaje1 > partido.puntaje2 and float(score1) > float(score2)) or \
           (partido.puntaje1 < partido.puntaje2 and float(score1) < float(score2)):
            ultimosPartidos.at[partido.Index, 'Acerto'] = 'Si'

    ultimosPartidos = ultimosPartidos[['Equipo 1', 'Resultado', 'Equipo 2', 'Score equipo 1', 'Score equipo 2', 'Acerto']]
    return ultimosPartidos

In [351]:
exportPath = 'cualitativo_sin_ultima_fecha.csv'
table = armarTabla(exportPath)
table.to_csv('cualitativo_CMM.csv', index=False)